# TDDFT with Psi4

Roberto Di Remigio ([@robertodr](https://github.com/robertodr))

[European National Competence Centre Sweden](https://enccs.se/)

## Using Psithon

The bridge to Psithon was built by Jeff Schriber ([@jeffschriber](https://github.com/jeffschriber))

- How does the input file look?
- How does the output file look?

## Using PsiAPI

- How can I write a script to run TDDFT?
- What variables are defined after a TDDFT run?

## Theory and implementation

For the Tamm-Dancoff approximation (TDA), the code uses a Davidson solver implemented by Ruhee D'Cunha ([@rdcunha](https://github.com/rdcunha)). For the random-phase approximation (RPA), the code uses a Hamiltonian solver implemented by Andrew James ([@amjames](https://github.com/amjames)). 
Both solvers are coded in Python and use the C++ implementation of the electronic Hessian-trial vector products implemented by Daniel Smith ([@dgasmith](https://https://github.com/dgasmith)).

## Plotting spectra

$$
\varepsilon(\omega) = 
      \frac{4\pi^{2}N_{\mathrm{A}}\omega}{3\times 1000\times \ln(10) (4 \pi \epsilon_{0}) n \hbar c}
      \sum_{i \rightarrow j}g_{ij}(\omega)|\mathbf{\mu}_{ij}|^{2}
$$

$$
\Delta\varepsilon(\omega) =
      \frac{16\pi^{2}N_{\mathrm{A}}\omega}{3\times 1000\times \ln(10) (4 \pi \epsilon_{0}) n \hbar c^{2}}
      \sum_{i \rightarrow j}g_{ij}(\omega)\Im(\mathbf{\mu}_{ij}\cdot\mathbf{m}_{ij})
$$

## Solvent effects

Functionality provided by Holger Kruse ([@hokru](https://github.com/hokru)) and Maximilian Scheurer ([@maxscheurer](https://github.com/maxscheurer))

In [2]:
import psi4

from psi4.driver.procrouting.response.scf_response import tdscf_excitations
from psi4.driver.p4util import spectrum

psi4.core.set_output_file("moxy.out")

# structure from Pedersen et al., CPL, submitted
moxy = psi4.geometry("""0 1
C  0.152133 -0.035800  0.485797
C -1.039475  0.615938 -0.061249
C  1.507144  0.097806 -0.148460
O -0.828215 -0.788248 -0.239431
H  0.153725 -0.249258  1.552136
H -1.863178  0.881921  0.593333
H -0.949807  1.214210 -0.962771
H  2.076806 -0.826189 -0.036671
H  2.074465  0.901788  0.325106
H  1.414895  0.315852 -1.212218
""", name="(S)-methyloxirane")

psi4.set_options({
    'save_jk': True,
})

method = 'HF'
basis = 'cc-pVDZ'
e, wfn = psi4.energy(f"{method}/{basis}", return_wfn=True, molecule=moxy)
res = tdscf_excitations(wfn, states=8)

from typing import Tuple, Dict

import numpy as np
import pandas as pd
import altair as alt

def plot_spectrum(data: Dict,
               *,
               title: str = "",
               x_title: Tuple[str, str] = ("ω", "au"),
               y_title: Tuple[str, str] = ("ε", "L⋅mol⁻¹⋅cm⁻¹"),
               offset: int = 0):
    hover = alt.selection_single(
      fields=["x"],
      nearest=True,
      on="mouseover",
      empty="none",
      clear="mouseout"
    )

    s1 = pd.DataFrame(data["convolution"])
    lines = alt.Chart(s1).mark_line(size=1.5).encode(
       x=alt.X("x", axis=alt.Axis(title=f"{x_title[0]} [{x_title[1]}]", offset=offset)),
       y=alt.Y("y", axis=alt.Axis(title=f"{y_title[0]} [{y_title[1]}]")),
       )

    points = lines.transform_filter(hover).mark_circle()

    tooltips = alt.Chart(s1).mark_rule().encode(
      x='x:Q',
      opacity=alt.condition(hover, alt.value(0.3), alt.value(0)),
      tooltip=[alt.Tooltip("x:Q", format=".4f", title=f"{x_title[0]}"), alt.Tooltip("y:Q", format=".1f", title=f"{y_title[0]}")]
      ).add_selection(
        hover
        )

    s2 = pd.DataFrame(data["sticks"])
    sticks = alt.Chart(s2).mark_bar(size=2, opacity=0.2, color="red").encode(
        x="poles:Q",
        y="residues:Q",
        )

    # Put the layers into a chart and bind the data
    plot = alt.layer(
      lines, points, tooltips, sticks,
      ).properties(
        title=title,
        )

    return plot

# get poles and residues to plot OPA and ECD spectra
poles = [r["EXCITATION ENERGY"] for r in res]
opa_residues = [np.linalg.norm(r["ELECTRIC DIPOLE TRANSITION MOMENT (LEN)"])**2 for r in res]
ecd_residues = [r["ROTATORY STRENGTH (LEN)"] for r in res]

opa_spectrum = spectrum(poles=poles, residues=opa_residues, gamma=0.01, out_units="nm")
opa_plot = plot_spectrum(opa_spectrum,
                         title="OPA (Gaussian broadening)",
                         x_title=("λ", "nm"))

ecd_spectrum = spectrum(poles=poles, residues=ecd_residues, kind="ECD", gamma=0.01, out_units="nm")
ecd_plot = plot_spectrum(ecd_spectrum,
                         title="ECD (Gaussian broadening)",
                         x_title=("λ", "nm"),
                         y_title=("Δε", "L⋅mol⁻¹⋅cm⁻¹"))

opa_plot & ecd_plot

alt.VConcatChart(...)